## Spark DataFrame Basics

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [2]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

In [3]:
df = spark.read.json("data/people.json")

In [4]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
# Shows the schema of the df
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
# Shows column names, we don't need to call it as it is a attribute.
df.columns

['age', 'name']

In [7]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   NULL|
| stddev|7.7781745930520225|   NULL|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



### How to define the schema

In [8]:
data_schema = [
    StructField("age", IntegerType(), True),
    StructField("name", StringType(), True)
]

In [9]:
final_struc = StructType(fields=data_schema)

In [10]:
df = spark.read.json("data/people.json", schema=final_struc)

In [11]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Selecting data from df

In [12]:
type(df["age"])

pyspark.sql.column.Column

In [13]:
df.select("age").show()

+----+
| age|
+----+
|NULL|
|  30|
|  19|
+----+



In [14]:
# Selecting the first two rows of the DF.
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [15]:
# Selecting multiple columns
df.select(["age", "name"]).show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [16]:
# Creating a new column
df.withColumn("double_age", df["age"]*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|NULL|Michael|      NULL|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [17]:
# Rename a column (not inplace)
df.withColumnRenamed("age", "my_new_age").show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      NULL|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



## Run SQL queries on spark dataframes

In [18]:
# Define a temporay view for spark to query from
df.createOrReplaceTempView("people")

In [19]:
results = spark.sql("SELECT * FROM people")
results.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [20]:
new_results = spark.sql("SELECT * FROM people WHERE age IS NOT NULL;")
new_results.show()

+---+------+
|age|  name|
+---+------+
| 30|  Andy|
| 19|Justin|
+---+------+

